In [42]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime
from datetime import timedelta
import re
import random
import math

In [2]:
# loading data
codes = pd.read_csv('../results/patients_w_codes.csv')

In [3]:
# version with dates into strings
codes2 = pd.read_csv('../results/patients_w_codes2.csv')

In [4]:
# to use a dict with the dates in the strings
def separate_date(df):
    df['first_event_date'] = 'NA'
    for i in range(len(df)):
        x = re.split(']', df['code'][i].replace('[', ''))
        df['first_event_date'][i] = x[0]
        df['code'][i] = x[1]
    return df

In [5]:
separate_date(codes2)
codes2

,id,code,first_event_date
0,005cad4958846409,B42A40A2H1F10A15B21B7B12I24I7F6B28B128A4D77B64...,2020-08-14
1,0065b57e5a9e784f,A37B234F4B4B8B2F2B2F3B4F0G3B2I8G17I215I82B9B23A,2020-03-26
2,006e5a1bf72e8b73,B56B7B35B11B3B11B10B13B14B8A178I21I123B6B5B6B5...,2020-06-02
3,009d11a2497f1d47,A334C142J,2020-10-14
4,00a5cf0620b6adfc,A201J213A,2020-10-26
...,...,...,...
3298,ff32e199c9482c20,B196B12B21B6F1B7B1I6B6A15B7B7I70B14B78B7B7B8B5...,2020-05-07
3299,ff46357762ea097e,B92B99B12B9B6B4B32F6I4B18I232C4A14B17I79D36B3B...,2020-04-20
3300,ff9318bd975b7050,A16A64B74B14A197I,2020-08-10
3301,ffabda76a8fbaecd,B24B182A12B13B7B6F1B0I7B1B5B7F1I56B15F218B4F10...,2020-05-05


In [6]:
# Function to split strings
def split_string(s):
    """
    s is a string code. Returns two lists, with letters (A) and numbers (B).
    """
    S = []
    for x in re.split('(\d+)', s):
        try:
            S.append(int(x))
        except ValueError:
            S.append(x)
    A, B = S[::2], S[1::2]
    return A, B

In [7]:
def case1(A, B, pos, neg):
    """
    Recognises case 1 reinfections (+|-|+).
    Added some lines to detect certain cases 2.
    """
    if A[0] in pos:
        i = 0
        c = B[i]
        # to find case 2 before case 1
        #if A[i] in pos and A[i+1] in pos and B[i] >= 90:
        #    return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        if A[i+1] in pos and c >= 90:
            return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        while A[i+1] not in neg:
            i += 1
            c += B[i]
            # to find case 2 before case 1
            if A[i+1] in pos and c >= 90:
                return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        # here the first neg is found
        i += 1
        c += B[i]
        while A[i+1] not in pos:
            i += 1
            c += B[i]
        if c >= 90:
            return 'Case 1', A[i+1:len(A)], B[i+1:len(B)]
        elif c >= 60:
            return 'Possible case 1', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return '', A[i+1:len(A)], B[i+1:len(B)]
    else:
        i = 0
        while A[i] not in pos:
            i += 1
        c = B[i]
        # same as above
        if A[i+1] in pos and c >= 90:
            return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        while A[i+1] not in neg:
            i += 1
            c += B[i]
            if A[i+1] in pos and c >= 90:
                return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        i += 1
        c += B[i]
        while A[i+1] not in pos:
            i += 1
            c += B[i]
        if c >= 90:
            return 'Case 1', A[i+1:len(A)], B[i+1:len(B)]
        elif c >= 60:
            return 'Possible case 1', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return '', A[i+1:len(A)], B[i+1:len(B)]

In [8]:
def case2(A, B, pos):
    """
    Recognises case 2 reinfections (+|+)
    """
    c = 0
    if A[0] in pos:
        for i in range(len(B)):
            if A[i+1] not in pos:
                if bool([x for x in A[i+1:len(A)] if(x in pos)]) == False:
                    if c >= 60:
                        return 'Possible Case 2', A[i:len(A)], B[i:len(B)]
                    else:
                        return '', A[i:len(A)], B[i:len(B)]
                c += B[i]
            else:
                if c + B[i] >= 90:
                    return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
                c = B[i]
        if c >= 60:
            return 'Possible Case 2', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return '', A[i+1:len(A)], B[i+1:len(B)]
    else:
        i = 0
        while A[i] not in pos:
            i += 1
        for j in range(i, len(B)):
            if A[j+1] not in pos:
                if bool([x for x in A[j+1:len(A)] if(x in pos)]) == False:
                    if c >= 60:
                        return 'Possible Case 2', A[j:len(A)], B[j:len(B)]
                    else:
                        return '', A[j:len(A)], B[j:len(B)]
                c += B[j]
            else:
                if c + B[j] >= 90:
                    return 'Case 2', A[j+1:len(A)], B[j+1:len(B)]
                c = B[j]
        if c >= 60:
            return 'Possible Case 2', A[j+1:len(A)], B[j+1:len(B)]
        else:
            return '', A[j+1:len(A)], B[j+1:len(B)]

In [9]:
# adding multiple reinfection management
def classif(s, pos = ['A', 'C', 'D'], neg = ['B']):
    """
    Takes a string and classifies it into reinfection cases 1 or 2
    """
    A, B = split_string(s)
    L = []
    while len(A) > 1:
        # indices stores a list between the first and last positive in the string
        indices = [i for i, x in enumerate(A) if x in pos]
        # b will be used to break the loop if there are no positives in the remaining string
        b = [x for x in A[1:len(A)] if(x in pos)]
        if any(n in A[min(indices):max(indices)] for n in neg):
            r, A, B = case1(A, B, pos, neg)
            if r != '':
                L.append(r)
            #print('Case 1', A)
        elif bool(b) == False:
            break
        else:
            r, A, B = case2(A, B, pos)
            #print('Case 2', A)
            if r != '':
                L.append(r)
    return L

In [10]:
# option for taking antigen tests into account
classif('B26A9B187F40F96F63F77A', pos = ['A', 'C', 'D', 'E'], neg = ['B', 'F'])

['Case 1']

In [12]:
# trying random strings
r = random.randint(0,3303)
print(codes.iloc[r]['id'], '\n')
s = codes.iloc[r]['code']

A, B = split_string(s)

print(s, '\n')

classif(s)

221767b6cbf6e2e6 

E1A6A3A25G3B58B62I112B18A1A5A4A48B 



['Case 1']

In [13]:
# applying function
d = {}
for x in codes['id']:
    d[str(x)] = classif(codes.loc[codes['id'] == x]['code'].values[0])

In [200]:
import csv

with open('../results/dict.csv', 'w') as f:
    for key in d.keys():
        f.write("%s, %s\n" % (key, d[key]))

In [14]:
# writing dictionary with strings and dates
D = {}
for x in codes2['id']:
    D[str(x)] = [codes2.loc[codes2['id'] == x]['first_event_date'].values[0],
                codes2.loc[codes2['id'] == x]['code'].values[0],
                classif(codes2.loc[codes2['id'] == x]['code'].values[0])]


In [289]:
with open('../results/dict2.csv', 'w') as f:
    for key in D.keys():
        f.write("%s, %s, %s, %s\n" % (key, D[key][0], D[key][1], D[key][2]))

In [15]:
N = list(d.values())
N = [str(x) for x in N]
freq = {}
for items in N:
    freq[items] = N.count(items)
freq

{"['Case 1']": 2109,
 "['Case 2']": 996,
 "['Case 2', 'Case 2']": 5,
 "['Case 1', 'Case 1']": 99,
 "['Possible case 1']": 26,
 "['Case 2', 'Case 1']": 17,
 '[]': 18,
 "['Case 1', 'Possible case 1']": 5,
 "['Possible case 1', 'Case 1']": 10,
 "['Case 1', 'Case 2']": 2,
 "['Possible Case 2']": 8,
 "['Case 1', 'Case 1', 'Possible case 1']": 2,
 "['Case 1', 'Possible case 1', 'Case 1']": 2,
 "['Case 2', 'Possible Case 2']": 1,
 "['Case 1', 'Case 1', 'Case 1']": 1,
 "['Possible case 1', 'Possible case 1']": 2}

In [16]:
# cases 1 
C1 = {}
for key in D.keys():
    if D[key][2] == ['Case 1']:
        C1[key] = D[key]

In [17]:
# function for selecting cases 1 with only two positives in their strings
def two_pos(S, pos = ['A', 'C', 'D']):
    l = []
    for p,c in enumerate(S):
        if(c in pos):
            l.append(p)
    if len(l) == 2:
        return True

In [18]:
FC1 = {}
for key in C1.keys():
    if two_pos(C1[key][1]):
        FC1[key] = C1[key]

In [19]:
len(FC1)

1235

In [20]:
# Discarding sequenced samples
def find_sequenced(s):
    if not 'C' in s:
        return 'Not sequenced'
    l = [x for x in s if(x in ['A', 'C', 'D'])]
    if l == ['C', 'C']:
        return 'Both sequenced'
    if l[0] == 'C':
        return 'First PCR sequenced'
    if l[1] == 'C':
        return 'Second PCR sequenced'

In [21]:
FC1_seq = {}
for k in FC1.keys():
    FC1_seq[k] = find_sequenced(FC1[k][1])

In [22]:
M = list(FC1_seq.values())
M = [str(x) for x in M]
freq = {}
for items in M:
    freq[items] = M.count(items)
freq

{'Not sequenced': 989,
 'Second PCR sequenced': 206,
 'First PCR sequenced': 32,
 'Both sequenced': 8}

In [23]:
# exploring no reinfections
NO = {}
for key in D.keys():
    if D[key][2] == []:
        NO[key] = D[key]

In [24]:
NO

{'16b35543f73d7ebb': ['2020-05-02',
  'B248I5B16I184N12A2B4A3B1A7Q28I50A6B3B7B1Q45F3B10B16B6B4B6B4B6B4B',
  []],
 '2492d31c709d9e90': ['2021-02-02', 'A15A7A6G132B140F10F53E', []],
 '26afdd6f52b296db': ['2020-09-11', 'B4A15A13A14A33A3G14A', []],
 '2f34cf46d4d5858a': ['2020-05-18',
  'H154B51B13B14B6H8B15B5A2U8A7A4G23J28G19A56B151B0J21B',
  []],
 '5316c9a74cdda114': ['2021-01-04',
  'A10A7A1G27I21I28A239B10J9B8B8B14B41B',
  []],
 '537e4ea8146251d5': ['2020-04-21',
  'B155B14B7A69F11I6A16I19B3B3B7B210I88B4B7B3B4B4B6B8B7B',
  []],
 '6acb778fc5a6bfa4': ['2020-09-22',
  'B33B6A10A18A24A40A41C26I21I40B93B22I',
  []],
 '8c02190c7d189ea4': ['2020-12-04',
  'N1Q0R2Q0S7B1Q1G262F9B5B5B11B16B30B11B3B4B2B2B2B1B1B2B4B3B3B17A13B6B7B8B8B13B2B56T',
  []],
 '902c5ff17c16857b': ['2020-11-16', 'A61A3A20G15C105I21I185J', []],
 'a7ced75a81d73d28': ['2021-02-05',
  'F185J5E12E0N0R28A11A4A7B3B32A1B2S1B22B1Q23A1A2A',
  []],
 'c0078581b6171547': ['2021-01-18', 'E28A45I15F5A23I26A170J', []],
 'c581972e22a1e830': 

In [288]:
# saving dict
with open('../results/filtered_dict.csv', 'w') as f:
    for key in FC1.keys():
        f.write("%s, %s, %s\n" % (key, FC1[key][0], FC1[key][1]))

In [25]:
# accounting for antigen tests
a = {}
for x in codes2['id']:
    a[str(x)] = classif(codes2.loc[codes2['id'] == x]['code'].values[0], 
                        pos = ['A', 'C', 'D', 'E'], neg = ['B', 'F'])

In [26]:
M = list(a.values())
M = [str(x) for x in M]
freq = {}
for items in M:
    freq[items] = M.count(items)
freq

{"['Case 1']": 2344,
 "['Case 2']": 652,
 "['Case 2', 'Case 1']": 47,
 "['Case 2', 'Case 2']": 6,
 "['Case 1', 'Case 1']": 158,
 "['Possible case 1']": 26,
 "['Case 1', 'Case 1', 'Case 1']": 4,
 '[]': 13,
 "['Case 1', 'Case 2']": 11,
 "['Case 1', 'Possible case 1']": 11,
 "['Possible case 1', 'Case 1']": 15,
 "['Case 1', 'Possible Case 2']": 2,
 "['Case 1', 'Case 1', 'Possible case 1']": 2,
 "['Case 1', 'Possible case 1', 'Case 1']": 2,
 "['Possible Case 2']": 6,
 "['Possible case 1', 'Case 1', 'Case 1']": 2,
 "['Possible case 1', 'Possible case 1']": 1,
 "['Possible case 1', 'Possible case 1', 'Case 1']": 1}

In [49]:
# FUNCTIONS FOR VARIANT ASSIGNMENT

# prealpha: start (2020-03-13) --> 2021-02-01
# alpha: 2021-02-08 --> 2021-06-21
# delta: 2021-06-28 --> 2021-12-13
# omicron: 2021-12-20 --> end (2022-05-21)

# function to obtain the last event date
def last_date(date, string):
    L, N = split_string(string)
    d = sum(N)
    return str(datetime.strptime(date, '%Y-%m-%d').date() + timedelta(days = d))

In [50]:
last_date(codes2['first_event_date'][0], codes2['code'][0])

'2022-03-15'

In [51]:
ld_list = []
for i in range(len(codes2)):
    ld_list.append(last_date(codes2['first_event_date'][i], codes2['code'][i]))

In [52]:
len(ld_list)

3303

In [53]:
# obtaining first and last date
print(min(codes2['first_event_date']))
print(max(ld_list))

2020-03-13
2022-05-21


In [ ]:
# we have to create a function able to extract the date of each infection